# 先跑一下这个论文中的模型先

In [1]:
from __future__ import print_function

import numpy as np
import pandas as pd
from keras.models import Model
from keras.utils import np_utils
import keras 
from keras.callbacks import ReduceLROnPlateau

Using TensorFlow backend.


In [2]:
def readucr(filename):
    data = np.loadtxt(filename, delimiter = ',')
    Y = data[:,0]
    X = data[:,1:]
    return X, Y
  
nb_epochs = 2000
nb_feature = 0
foder = u"E:/Jupyter File/UCR_TS_Archive_2015/"

#flist = ['Adiac', 'Beef', 'CBF', 'ChlorineConcentration', 'CinC_ECG_torso', 'Coffee', 'Cricket_X', 'Cricket_Y', 'Cricket_Z', 
#'DiatomSizeReduction', 'ECGFiveDays', 'FaceAll', 'FaceFour', 'FacesUCR', '50words', 'FISH', 'Gun_Point', 'Haptics', 
#'InlineSkate', 'ItalyPowerDemand', 'Lighting2', 'Lighting7', 'MALLAT', 'MedicalImages', 'MoteStrain', 'NonInvasiveFatalECG_Thorax1', 
#'NonInvasiveFatalECG_Thorax2', 'OliveOil', 'OSULeaf', 'SonyAIBORobotSurface', 'SonyAIBORobotSurfaceII', 'StarLightCurves', 'SwedishLeaf', 'Symbols', 
#'synthetic_control', 'Trace', 'TwoLeadECG', 'Two_Patterns', 'uWaveGestureLibrary_X', 'uWaveGestureLibrary_Y', 'uWaveGestureLibrary_Z', 'wafer', 'WordsSynonyms', 'yoga']

#flist  = ['MiddlePhalanxTW']
#for each in flist:
each = 'Adiac'

fname = each
x_train, y_train = readucr(foder+fname+'/'+fname+'_TRAIN')
x_test, y_test = readucr(foder+fname+'/'+fname+'_TEST')
nb_classes = len(np.unique(y_test))
nb_feature = x_train.shape[1]
batch_size = min(x_train.shape[0]/10, 16)

y_train = (y_train - y_train.min())/(y_train.max()-y_train.min())*(nb_classes-1)
y_test = (y_test - y_test.min())/(y_test.max()-y_test.min())*(nb_classes-1)
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

x_train_mean = x_train.mean()
x_train_std = x_train.std()
x_train = (x_train - x_train_mean)/(x_train_std) 
x_test = (x_test - x_train_mean)/(x_train_std)

print("############### raw data")
print("# x_train.shape: ",x_train.shape)
print("# x_test.shape: ",x_test.shape)

print("# Y_train.shape: ",Y_train.shape)
print("# Y_test.shape: ",Y_test.shape)

############### raw data
# x_train.shape:  (390, 176)
# x_test.shape:  (391, 176)
# Y_train.shape:  (390, 37)
# Y_test.shape:  (391, 37)


## 以上是 raw data
#### 以下先添加 LSTM 的 input格式

In [3]:
'''使用LSTM'''
# 先整理一下数据格式
timesteps = 1
x_train = x_train.reshape(x_train.shape[0], timesteps, x_train.shape[1])
x_test = x_test.reshape(x_test.shape[0], timesteps, x_test.shape[1])

print("# x_train.shape: ",x_train.shape)
print("# x_test.shape: ",x_test.shape)

print("# Y_train.shape: ",Y_train.shape)
print("# Y_test.shape: ",Y_test.shape)

# x_train.shape:  (390, 1, 176)
# x_test.shape:  (391, 1, 176)
# Y_train.shape:  (390, 37)
# Y_test.shape:  (391, 37)


#### model

In [6]:
from keras.models import Model
from keras.layers import Input, PReLU, Dense, LSTM, multiply, concatenate, Activation
from keras.layers import Conv1D, BatchNormalization, GlobalAveragePooling1D, Permute, Dropout

MAX_SEQUENCE_LENGTH_LIST = 176
ip = Input(shape=(1, MAX_SEQUENCE_LENGTH_LIST))  #  timesteps, nb_feature/length_sequence

x = LSTM(8)(ip)
x = Dropout(0.8)(x)
y = Permute((2, 1))(ip)
y = Conv1D(128, 8, padding='same', kernel_initializer='he_uniform')(y)
y = BatchNormalization()(y)
y = Activation('relu')(y)

y = Conv1D(256, 5, padding='same', kernel_initializer='he_uniform')(y)
y = BatchNormalization()(y)
y = Activation('relu')(y)

y = Conv1D(128, 3, padding='same', kernel_initializer='he_uniform')(y)
y = BatchNormalization()(y)
y = Activation('relu')(y)

y = GlobalAveragePooling1D()(y)

x = concatenate([x, y])

out = Dense(nb_classes, activation='softmax')(x)

model = Model(ip, out)

model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_2 (InputLayer)             (None, 1, 176)        0                                            
____________________________________________________________________________________________________
permute_2 (Permute)              (None, 176, 1)        0           input_2[0][0]                    
____________________________________________________________________________________________________
conv1d_4 (Conv1D)                (None, 176, 128)      1152        permute_2[0][0]                  
____________________________________________________________________________________________________
batch_normalization_4 (BatchNorm (None, 176, 128)      512         conv1d_4[0][0]                   
___________________________________________________________________________________________

# train and evaluate model

In [9]:
from keras.models import Model
from keras.layers import Permute
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.wrappers.scikit_learn import KerasClassifier


reduce_lr = ReduceLROnPlateau(monitor='val_acc', patience=100, mode='max', cooldown=0, min_lr=1e-4, verbose=2)

learning_rate=1e-3
optm = Adam(lr=learning_rate)

model.compile(optimizer=optm, loss='categorical_crossentropy', metrics=['accuracy'])

hist = model.fit(x_train, Y_train, batch_size=batch_size, epochs=nb_epochs, validation_data=(x_test, Y_test))

#Print the testing results which has the lowest training loss.
log = pd.DataFrame(hist.history)
print('\n','min_loss: ',log.loc[log['loss'].idxmin]['loss'], 'val_acc: ',log.loc[log['loss'].idxmin]['val_acc'])

Train on 390 samples, validate on 391 samples
Epoch 1/2000
390/390 [==============================] - 5s - loss: 3.6298 - acc: 0.0410 - val_loss: 4.1425 - val_acc: 0.0179
Epoch 2/2000
390/390 [==============================] - 3s - loss: 3.4553 - acc: 0.1077 - val_loss: 3.9110 - val_acc: 0.0179
Epoch 3/2000
390/390 [==============================] - 3s - loss: 3.3091 - acc: 0.1256 - val_loss: 3.9346 - val_acc: 0.0409
Epoch 4/2000
390/390 [==============================] - 3s - loss: 3.2041 - acc: 0.1231 - val_loss: 3.9218 - val_acc: 0.0512
Epoch 5/2000
390/390 [==============================] - 3s - loss: 3.0722 - acc: 0.1949 - val_loss: 3.7549 - val_acc: 0.0563
Epoch 6/2000
390/390 [==============================] - 3s - loss: 2.9548 - acc: 0.2256 - val_loss: 3.5664 - val_acc: 0.0665
Epoch 7/2000
390/390 [==============================] - 3s - loss: 2.8594 - acc: 0.2462 - val_loss: 3.4110 - val_acc: 0.0793
Epoch 8/2000
390/390 [==============================] - 3s - loss: 2.8160 - acc

390/390 [==============================] - 4s - loss: 1.0842 - acc: 0.7000 - val_loss: 1.1090 - val_acc: 0.6854
Epoch 66/2000
390/390 [==============================] - 3s - loss: 1.0238 - acc: 0.7103 - val_loss: 1.4543 - val_acc: 0.5243
Epoch 67/2000
390/390 [==============================] - 3s - loss: 1.0445 - acc: 0.7385 - val_loss: 1.1828 - val_acc: 0.5985
Epoch 68/2000
390/390 [==============================] - 3s - loss: 1.0564 - acc: 0.7333 - val_loss: 1.3323 - val_acc: 0.5908
Epoch 69/2000
390/390 [==============================] - 3s - loss: 1.0125 - acc: 0.7128 - val_loss: 1.1047 - val_acc: 0.6394
Epoch 70/2000
390/390 [==============================] - 3s - loss: 1.0009 - acc: 0.7308 - val_loss: 1.1673 - val_acc: 0.6394
Epoch 71/2000
390/390 [==============================] - 3s - loss: 1.0525 - acc: 0.7179 - val_loss: 1.4249 - val_acc: 0.5217
Epoch 72/2000
390/390 [==============================] - 3s - loss: 1.0551 - acc: 0.7205 - val_loss: 1.0199 - val_acc: 0.7263
Epoch 

390/390 [==============================] - 3s - loss: 0.6114 - acc: 0.8308 - val_loss: 1.1795 - val_acc: 0.5934
Epoch 130/2000
390/390 [==============================] - 3s - loss: 0.6340 - acc: 0.7949 - val_loss: 1.3910 - val_acc: 0.5320
Epoch 131/2000
390/390 [==============================] - 3s - loss: 0.6595 - acc: 0.8154 - val_loss: 0.9354 - val_acc: 0.7008
Epoch 132/2000
390/390 [==============================] - 3s - loss: 0.6737 - acc: 0.7923 - val_loss: 1.0695 - val_acc: 0.6547
Epoch 133/2000
390/390 [==============================] - 3s - loss: 0.6980 - acc: 0.7923 - val_loss: 1.1638 - val_acc: 0.6138
Epoch 134/2000
390/390 [==============================] - 3s - loss: 0.6646 - acc: 0.8026 - val_loss: 0.9109 - val_acc: 0.6982
Epoch 135/2000
390/390 [==============================] - 3s - loss: 0.6490 - acc: 0.8103 - val_loss: 2.0336 - val_acc: 0.4834
Epoch 136/2000
390/390 [==============================] - 3s - loss: 0.6759 - acc: 0.7821 - val_loss: 0.7472 - val_acc: 0.7647

390/390 [==============================] - 3s - loss: 0.4345 - acc: 0.8538 - val_loss: 0.8975 - val_acc: 0.6777
Epoch 194/2000
390/390 [==============================] - 3s - loss: 0.4685 - acc: 0.8462 - val_loss: 0.7974 - val_acc: 0.7442
Epoch 195/2000
390/390 [==============================] - 3s - loss: 0.4155 - acc: 0.8615 - val_loss: 0.7800 - val_acc: 0.7238
Epoch 196/2000
390/390 [==============================] - 3s - loss: 0.4339 - acc: 0.8615 - val_loss: 0.8723 - val_acc: 0.7212
Epoch 197/2000
390/390 [==============================] - 3s - loss: 0.4659 - acc: 0.8462 - val_loss: 1.5369 - val_acc: 0.4962
Epoch 198/2000
390/390 [==============================] - 3s - loss: 0.4557 - acc: 0.8256 - val_loss: 0.9313 - val_acc: 0.7084
Epoch 199/2000
390/390 [==============================] - 3s - loss: 0.4625 - acc: 0.8641 - val_loss: 0.9046 - val_acc: 0.7417
Epoch 200/2000
390/390 [==============================] - 3s - loss: 0.4834 - acc: 0.8538 - val_loss: 1.3127 - val_acc: 0.5882

390/390 [==============================] - 3s - loss: 0.3294 - acc: 0.9026 - val_loss: 1.6746 - val_acc: 0.5371
Epoch 258/2000
390/390 [==============================] - 3s - loss: 0.3407 - acc: 0.8795 - val_loss: 1.2163 - val_acc: 0.6215
Epoch 259/2000
390/390 [==============================] - 3s - loss: 0.3556 - acc: 0.8974 - val_loss: 0.9984 - val_acc: 0.6650
Epoch 260/2000
390/390 [==============================] - 3s - loss: 0.4527 - acc: 0.8538 - val_loss: 3.3037 - val_acc: 0.4373
Epoch 261/2000
390/390 [==============================] - 3s - loss: 0.3902 - acc: 0.8769 - val_loss: 1.8390 - val_acc: 0.4885
Epoch 262/2000
390/390 [==============================] - 3s - loss: 0.3614 - acc: 0.8923 - val_loss: 0.9862 - val_acc: 0.6624
Epoch 263/2000
390/390 [==============================] - 3s - loss: 0.3456 - acc: 0.8974 - val_loss: 0.9518 - val_acc: 0.6880
Epoch 264/2000
390/390 [==============================] - 3s - loss: 0.3948 - acc: 0.8538 - val_loss: 1.2337 - val_acc: 0.5882

...

390/390 [==============================] - 3s - loss: 0.0507 - acc: 0.9897 - val_loss: 0.8338 - val_acc: 0.7903
Epoch 1730/2000
390/390 [==============================] - 3s - loss: 0.0172 - acc: 0.9974 - val_loss: 0.8040 - val_acc: 0.8082
Epoch 1731/2000
390/390 [==============================] - 3s - loss: 0.0178 - acc: 0.9974 - val_loss: 0.9199 - val_acc: 0.7673
Epoch 1732/2000
390/390 [==============================] - 3s - loss: 0.0223 - acc: 0.9974 - val_loss: 1.0809 - val_acc: 0.7417
Epoch 1733/2000
390/390 [==============================] - 3s - loss: 0.0168 - acc: 0.9949 - val_loss: 1.0698 - val_acc: 0.7621
Epoch 1734/2000
390/390 [==============================] - 3s - loss: 0.0161 - acc: 0.9949 - val_loss: 0.9644 - val_acc: 0.7698
Epoch 1735/2000
390/390 [==============================] - 3s - loss: 0.0260 - acc: 0.9897 - val_loss: 0.5925 - val_acc: 0.8465
Epoch 1736/2000
390/390 [==============================] - 3s - loss: 0.0282 - acc: 0.9923 - val_loss: 0.6567 - val_acc:

390/390 [==============================] - 3s - loss: 0.0422 - acc: 0.9897 - val_loss: 0.8345 - val_acc: 0.8005
Epoch 1794/2000
390/390 [==============================] - 3s - loss: 0.0230 - acc: 0.9949 - val_loss: 0.7691 - val_acc: 0.7903
Epoch 1795/2000
390/390 [==============================] - 3s - loss: 0.0158 - acc: 1.0000 - val_loss: 0.6905 - val_acc: 0.8159
Epoch 1796/2000
390/390 [==============================] - 3s - loss: 0.0150 - acc: 0.9974 - val_loss: 0.6574 - val_acc: 0.8159
Epoch 1797/2000
390/390 [==============================] - 3s - loss: 0.0147 - acc: 1.0000 - val_loss: 0.6066 - val_acc: 0.8363
Epoch 1798/2000
390/390 [==============================] - 3s - loss: 0.0124 - acc: 1.0000 - val_loss: 0.5594 - val_acc: 0.8312
Epoch 1799/2000
390/390 [==============================] - 3s - loss: 0.0143 - acc: 0.9974 - val_loss: 0.5935 - val_acc: 0.8465
Epoch 1800/2000
390/390 [==============================] - 3s - loss: 0.0185 - acc: 0.9949 - val_loss: 0.6821 - val_acc:

390/390 [==============================] - 3s - loss: 0.0116 - acc: 0.9974 - val_loss: 0.6428 - val_acc: 0.8440
Epoch 1858/2000
390/390 [==============================] - 3s - loss: 0.0135 - acc: 0.9974 - val_loss: 0.7093 - val_acc: 0.8312
Epoch 1859/2000
390/390 [==============================] - 3s - loss: 0.0087 - acc: 1.0000 - val_loss: 0.6419 - val_acc: 0.8286
Epoch 1860/2000
390/390 [==============================] - 3s - loss: 0.0192 - acc: 0.9974 - val_loss: 0.7327 - val_acc: 0.8312
Epoch 1861/2000
390/390 [==============================] - 3s - loss: 0.0174 - acc: 0.9949 - val_loss: 1.4674 - val_acc: 0.6905
Epoch 1862/2000
390/390 [==============================] - 3s - loss: 0.0145 - acc: 1.0000 - val_loss: 0.6140 - val_acc: 0.8542
Epoch 1863/2000
390/390 [==============================] - 3s - loss: 0.0145 - acc: 0.9974 - val_loss: 0.6471 - val_acc: 0.8491
Epoch 1864/2000
390/390 [==============================] - 3s - loss: 0.0147 - acc: 0.9974 - val_loss: 0.6235 - val_acc:

390/390 [==============================] - 3s - loss: 0.0150 - acc: 0.9974 - val_loss: 0.5920 - val_acc: 0.8465
Epoch 1922/2000
390/390 [==============================] - 3s - loss: 0.0087 - acc: 1.0000 - val_loss: 0.6291 - val_acc: 0.8542
Epoch 1923/2000
390/390 [==============================] - 3s - loss: 0.0105 - acc: 1.0000 - val_loss: 0.5748 - val_acc: 0.8517
Epoch 1924/2000
390/390 [==============================] - 3s - loss: 0.0098 - acc: 1.0000 - val_loss: 0.5985 - val_acc: 0.8593
Epoch 1925/2000
390/390 [==============================] - 3s - loss: 0.0076 - acc: 1.0000 - val_loss: 0.6028 - val_acc: 0.8645
Epoch 1926/2000
390/390 [==============================] - 3s - loss: 0.0068 - acc: 1.0000 - val_loss: 0.6847 - val_acc: 0.8338
Epoch 1927/2000
390/390 [==============================] - 3s - loss: 0.0071 - acc: 1.0000 - val_loss: 0.6542 - val_acc: 0.8312
Epoch 1928/2000
390/390 [==============================] - 3s - loss: 0.0044 - acc: 1.0000 - val_loss: 0.6236 - val_acc:

390/390 [==============================] - 3s - loss: 0.0068 - acc: 1.0000 - val_loss: 0.6612 - val_acc: 0.8363
Epoch 1986/2000
390/390 [==============================] - 3s - loss: 0.0089 - acc: 1.0000 - val_loss: 0.6787 - val_acc: 0.8184
Epoch 1987/2000
390/390 [==============================] - 3s - loss: 0.0064 - acc: 1.0000 - val_loss: 0.6443 - val_acc: 0.8414
Epoch 1988/2000
390/390 [==============================] - 3s - loss: 0.0062 - acc: 1.0000 - val_loss: 0.6610 - val_acc: 0.8363
Epoch 1989/2000
390/390 [==============================] - 3s - loss: 0.0074 - acc: 1.0000 - val_loss: 0.6660 - val_acc: 0.8440
Epoch 1990/2000
390/390 [==============================] - 3s - loss: 0.0077 - acc: 1.0000 - val_loss: 0.6694 - val_acc: 0.8389
Epoch 1991/2000
390/390 [==============================] - 3s - loss: 0.0063 - acc: 1.0000 - val_loss: 0.6199 - val_acc: 0.8491
Epoch 1992/2000
390/390 [==============================] - 3s - loss: 0.0058 - acc: 1.0000 - val_loss: 0.6815 - val_acc:

In [ ]:
'''CNN 所需要的格式'''
# (390, 176, 1, 1)
x_train = x_train.reshape(x_train.shape + (1,1,))
x_test = x_test.reshape(x_test.shape + (1,1,))

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='val_acc', patience=100, mode='max', factor=factor, cooldown=0, min_lr=1e-4, verbose=2)

callback_list = [model_checkpoint, reduce_lr]

optm = Adam(lr=learning_rate)

model.compile(optimizer=optm, loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, batch_size=batch_size, epochs=nb_epochs, callbacks=callback_list,
              class_weight=class_weight, verbose=2, validation_data=(X_test, y_test))

In [ ]:
optm = Adam(lr=1e-3)

model.compile(optimizer=optm, loss='categorical_crossentropy', metrics=['accuracy'])

print("\nEvaluating : ")

loss, accuracy = model.evaluate(X_test, y_test, batch_size=batch_size)

print()
print("Final Accuracy : ", accuracy)

In [ ]:
print("# x_train.shape: ",x_train.shape)
print("# x_test.shape: ",x_test.shape)

print("# Y_train.shape: ",Y_train.shape)
print("# Y_test.shape: ",Y_test.shape)